In [1]:
# Import pandas and sqlite3

import pandas as pd
import sqlite3

**https://data.louisvilleky.gov/**

**https://data.louisvilleky.gov/datasets/a220289a40c945298d7f9d5c8dc7b3c0_0/explore**

In [2]:
# Create a dataframe from the .csv obtained from the address above

data = pd.read_csv('Louisville_Metro_KY_-_Crime_Data_2024.csv')

data.head(3)

,incident_number,date_reported,date_occurred,badge_id,offense_classification,offense_code_name,nibrs_code,nibrs_group_name,was_offense_completed,lmpd_division,lmpd_beat,location_category,block_address,city,zip_code,ObjectId
0,LMPD24056997,2024/05/17 20:32:00+00,2024/04/14 21:49:00+00,5314.0,11 SIMPLE ASSAULT,ASSAULT - 4TH DEGREE (DOMESTIC VIOLENCE) MINOR...,13B,A,YES,7TH DIVISION,735,RESIDENCE/HOME,4600 BLOCK LOR ANN AVE,LOUISVILLE,40219,1
1,LMPD24054715,2024/05/11 12:37:00+00,2024/05/11 03:00:00+00,8358.0,21 THEFT FR VEH,TBUT OR DISP CONTENTS FROM VEH 514.030 24140 23F,23F,A,YES,7TH DIVISION,723,PARKING/ DROP LOT/ GARAGE,7600 BLOCK MOUNTAIN VIEW CIR,LOUISVILLE,40228,2
2,LMPD24056995,2024/05/17 20:19:00+00,2024/05/05 06:00:00+00,7166.0,9 AGGRAVATED ASSAULT,ASSAULT - 1ST DEGREE 508.010 13150 13A,13A,A,YES,3RD DIVISION,315,BAR/NIGHTCLUB,8200 BLOCK DIXIE HWY,LOUISVILLE,40202,3


In [3]:
# Overwrite was_offense_completed column with 1,0 versions of the given 'Boolean' 'YES':'NO'

data['was_offense_completed'] = data.was_offense_completed.map({'YES':1, 'NO':0})

In [4]:
# Rename ID column to a format more friendly to our SQL conversion

data = data.rename(columns={'ObjectId':'object_id'})

In [5]:
# Isolate dates and times into separate columns from the string 'datetime' values given in the .csv for both date_reported and
# date_occurred

# Coerce the values in date_reported into proper datetime objects

data['date_reported'] = pd.to_datetime(data['date_reported'])

# Isolate the time portion of date_reported and save it to the 'time_reported' column

data['time_reported'] = data['date_reported'].dt.time

# Isolate the date portion of date_reported and save it to the 'date_reported' column

data['date_reported'] = data['date_reported'].dt.date

# Coerce the values in date_occurred into proper datetime objects

data['date_occurred'] = pd.to_datetime(data['date_occurred'])

# Isolate the time portion of date_occurred and save it to the 'time_occurred' column

data['time_occurred'] = data['date_occurred'].dt.time

# Isolate the date portion of date_occurred and save it to the 'date_occurred' column

data['date_occurred'] = data['date_occurred'].dt.date

In [6]:
# Rearrange column names into the desired order

data = data.reindex(columns=['object_id', 'incident_number', 'date_reported', 'time_reported','date_occurred', 'time_occurred', 
                         'badge_id', 'offense_classification', 'offense_code_name', 'nibrs_code', 'nibrs_group_name', 
                         'was_offense_completed', 'lmpd_division', 'lmpd_beat', 'location_category', 'block_address', 
                         'city', 'zip_code'])

In [7]:
# Return all unique values from the zip_code column. Note not all values are 5 digit zip codes, but a variety of formats

data['zip_code'].unique()

array(['40219', '40228', '40202', '40291', '40214', '40258', '40208',
       '40241', '40059', '40204', '40216', '40213', '40245', '40212',
       '40272', '40118', '40206', '40218', '40210', '40211', '40203',
       '40222', '40229', '40220', '40209', '40215', '40205', '40217',
       '40223', '40299', '40018', '40207', '40023', '40243', nan, '99999',
       '40242', '40241 0000', '40241-0000', '40177', '40508',
       '40245 0000', '40258 0000', '40233', '402120000', '47130',
       '402190000', '40206 0000', '40218 0000', '40206-0000',
       '40208 0000', '40041', '40292', '40215 0000', '40216 0000',
       '40214 0000', '30349', '40291-0000', '40215-0000', '40203 0000',
       '40211 0000', '40213-0000', '40025', '40225', '40272 0000',
       '402140000', '40291 0000', '40212-0000', '40165', '40210 0000',
       '40219 0000', '40212 0000', '40205 0000', '40214-0000',
       '40229-0000', '402130000', '40118-0000', '40204-0000',
       '40229 0000', '47150', '40242 0000', '40258000

In [8]:
# Overwrite the zip_code column with the first 5 characters of the previous value as a string

data['zip_code'] = data['zip_code'].str[:5]

In [9]:
# Confirm that we have properly coerced 5 digit zip codes

data['zip_code'].unique()

array(['40219', '40228', '40202', '40291', '40214', '40258', '40208',
       '40241', '40059', '40204', '40216', '40213', '40245', '40212',
       '40272', '40118', '40206', '40218', '40210', '40211', '40203',
       '40222', '40229', '40220', '40209', '40215', '40205', '40217',
       '40223', '40299', '40018', '40207', '40023', '40243', nan, '99999',
       '40242', '40177', '40508', '40233', '47130', '40041', '40292',
       '30349', '40025', '40225', '40165', '47150', '46220', '41031',
       '42728', '40014'], dtype=object)

In [10]:
# Inspect the DataFrame to confirm that our transformations have executed as expected

data.head(3)

,object_id,incident_number,date_reported,time_reported,date_occurred,time_occurred,badge_id,offense_classification,offense_code_name,nibrs_code,nibrs_group_name,was_offense_completed,lmpd_division,lmpd_beat,location_category,block_address,city,zip_code
0,1,LMPD24056997,2024-05-17,20:32:00,2024-04-14,21:49:00,5314.0,11 SIMPLE ASSAULT,ASSAULT - 4TH DEGREE (DOMESTIC VIOLENCE) MINOR...,13B,A,1,7TH DIVISION,735,RESIDENCE/HOME,4600 BLOCK LOR ANN AVE,LOUISVILLE,40219
1,2,LMPD24054715,2024-05-11,12:37:00,2024-05-11,03:00:00,8358.0,21 THEFT FR VEH,TBUT OR DISP CONTENTS FROM VEH 514.030 24140 23F,23F,A,1,7TH DIVISION,723,PARKING/ DROP LOT/ GARAGE,7600 BLOCK MOUNTAIN VIEW CIR,LOUISVILLE,40228
2,3,LMPD24056995,2024-05-17,20:19:00,2024-05-05,06:00:00,7166.0,9 AGGRAVATED ASSAULT,ASSAULT - 1ST DEGREE 508.010 13150 13A,13A,A,1,3RD DIVISION,315,BAR/NIGHTCLUB,8200 BLOCK DIXIE HWY,LOUISVILLE,40202


In [11]:
#Open connection and drop tables if it exist (from previous runs)

conn = sqlite3.connect('crime.db')

# Drop Crime table if exists

conn.execute('DROP TABLE IF EXISTS CRIME')

# Drop Date table if exists

conn.execute('DROP TABLE IF EXISTS DATE')

# Drop Officer table if exists

conn.execute('DROP TABLE IF EXISTS OFFICER')

# Drop Location table if exists

conn.execute('DROP TABLE IF EXISTS LOCATION')

**https://www.sqlite.org/datatype3.html**

In [12]:
# Use connection to create a table CRIME to store a subset of the data from our .csv

conn.execute('''
         CREATE TABLE IF NOT EXISTS CRIME 
         (OBJECT_ID INT PRIMARY KEY NOT NULL,
         INCIDENT_NUMBER TEXT,
         OFFENSE_CLASSIFICATION TEXT,
         OFFENSE_CODE_NAME TEXT,
         NIBRS_CODE TEXT,
         NIBRS_GROUP_NAME TEXT,
         WAS_OFFENSE_COMPLETED INT);''')

In [13]:
# Use connection to create a table DATE to store a subset of the data from our .csv

conn.execute('''
        CREATE TABLE IF NOT EXISTS DATE
        (DATE_ID INTEGER PRIMARY KEY AUTOINCREMENT,
        DATE_REPORTED TEXT,
        TIME_REPORTED TEXT,
        DATE_OCCURRED TEXT,
        TIME_OCCURRED TEXT,
        OBJECT_ID INT,
        FOREIGN KEY (OBJECT_ID) REFERENCES CRIME (OBJECT_ID))
        ''')

In [14]:
# Use connection to create a table OFFICER to store a subset of the data from our .csv

conn.execute('''
        CREATE TABLE IF NOT EXISTS OFFICER
        (OFFICER_ID INTEGER PRIMARY KEY AUTOINCREMENT,
        BADGE_ID INT,
        LMPD_DIVISION TEXT,
        LMPD_BEAT INT,
        OBJECT_ID INT,
        FOREIGN KEY (OBJECT_ID) REFERENCES CRIME (OBJECT_ID))
        ''')

In [15]:
# Use connection to create a table LOCATION to store a subset of the data from our .csv

conn.execute('''
        CREATE TABLE IF NOT EXISTS LOCATION
        (LOCATION_ID INTEGER PRIMARY KEY AUTOINCREMENT,
        LOCATION_CATEGORY TEXT,
        BLOCK_ADDRESS TEXT,
        CITY TEXT,
        ZIP_CODE INT,
        OBJECT_ID INT,
        FOREIGN KEY (OBJECT_ID) REFERENCES CRIME (OBJECT_ID))
        ''')

In [16]:
# Use our DataFrame to push a subset of the data it contains to the CRIME table that we just created 

data[['object_id', 'incident_number', 'offense_classification', 'offense_code_name', 'nibrs_code', 'nibrs_group_name', 
     'was_offense_completed']].to_sql("CRIME", conn, if_exists='append', index=False)

22904

In [17]:
# Use our DataFrame to push a subset of the data it contains to the DATE table that we just created  

data[['object_id', 'date_reported', 'time_reported', 'date_occurred', 'time_occurred']].to_sql("DATE", conn, 
                                                                                               if_exists='append', index=False)

22904

In [18]:
# Use our DataFrame to push a subset of the data it contains to the OFFICER table that we just created 

data[['object_id', 'badge_id', 'lmpd_division', 'lmpd_beat']].to_sql("OFFICER", conn, 
                                                                                               if_exists='append', index=False)

22904

In [19]:
# Use our DataFrame to push a subset of the data it contains to the LOCATION table that we just created 

data[['object_id', 'location_category', 'block_address', 'city', 'zip_code']].to_sql("LOCATION", conn, 
                                                                                               if_exists='append', index=False)

22904

# To browse this database I've downloaded a tool from:

**https://sqlitebrowser.org/**

In [20]:
# Create a query to isolate crime data for the zip code 40202 limited to 100 rows

query_40202 = """SELECT CRIME.OBJECT_ID, DATE.DATE_OCCURRED, CRIME.OFFENSE_CLASSIFICATION, LOCATION.ZIP_CODE, 
                 LOCATION.BLOCK_ADDRESS, OFFICER.BADGE_ID
                 FROM CRIME 
                 INNER JOIN DATE ON CRIME.OBJECT_ID = DATE.OBJECT_ID 
                 INNER JOIN OFFICER ON CRIME.OBJECT_ID = OFFICER.OBJECT_ID
                 INNER JOIN LOCATION ON CRIME.OBJECT_ID = LOCATION.OBJECT_ID
                 WHERE ZIP_CODE = 40202 LIMIT 100"""

In [21]:
# Utilize pd.read_sql_query to execute the query passing our query_40202 variable as a string and the connection object to the
# database file

df_40202 = pd.read_sql_query(query_40202, conn)

In [22]:
# Inspect the DataFrame to ensure this has worked as expected

df_40202

,OBJECT_ID,DATE_OCCURRED,OFFENSE_CLASSIFICATION,ZIP_CODE,BLOCK_ADDRESS,BADGE_ID
0,3,2024-05-05,9 AGGRAVATED ASSAULT,40202,8200 BLOCK DIXIE HWY,7166
1,82,2024-05-11,8 ROBBERY,40202,700 BLOCK MARSHALL ST,5512
2,89,2024-05-11,56 ALL OTHER OFFENSES,40202,400 BLOCK S 3RD ST,5407
3,90,2024-05-11,51 DISORDERLY CONDUCT,40202,300 BLOCK W MUHAMMAD ALI BLVD,5407
4,91,2024-05-11,52 DRUNKENNESS,40202,400 BLOCK S 3RD ST,5407
...,...,...,...,...,...,...
95,1926,2024-05-03,24 VANDALISM,40202,800 BLOCK W BROADWAY,8173
96,1931,2024-05-03,21 THEFT FR VEH,40202,500 BLOCK W BROADWAY,8620
97,1947,2024-05-03,19 THEFT FR BLDG,40202,400 BLOCK W MARKET ST,8515
98,1951,2024-05-02,12 INTIMIDATION,40202,400 BLOCK S 2ND ST,5135


In [23]:
# Create a query to return the first 5 rows from the database file

query_all = "SELECT * FROM CRIME LIMIT 5"

# Read our query into a DataFrame

data_peek = pd.read_sql_query(query_all, conn)

# Inspect our CRIME table

data_peek

,OBJECT_ID,INCIDENT_NUMBER,OFFENSE_CLASSIFICATION,OFFENSE_CODE_NAME,NIBRS_CODE,NIBRS_GROUP_NAME,WAS_OFFENSE_COMPLETED
0,1,LMPD24056997,11 SIMPLE ASSAULT,ASSAULT - 4TH DEGREE (DOMESTIC VIOLENCE) MINOR...,13B,A,1
1,2,LMPD24054715,21 THEFT FR VEH,TBUT OR DISP CONTENTS FROM VEH 514.030 24140 23F,23F,A,1
2,3,LMPD24056995,9 AGGRAVATED ASSAULT,ASSAULT - 1ST DEGREE 508.010 13150 13A,13A,A,1
3,4,LMPD24056921,28 FRAUD CREDIT CARD/ATM,FRAUDULENT USE OF A CREDIT CARD $500 < $1000 4...,26B,A,1
4,5,LMPD24054714,21 THEFT FR VEH,TBUT OR DISP CONTENTS FROM VEH 514.030 24140 23F,23F,A,0


In [24]:
# Close our connection to the database to prevent database locking 

conn.close()